In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading data

In [2]:
import pandas as pd
external_mlqa = pd.read_csv('../input/mlqa-hindi-processed/mlqa_hindi.csv')
external_xquad = pd.read_csv('../input/mlqa-hindi-processed/xquad.csv')
external_train = pd.concat([external_mlqa, external_xquad])#[:64]
print(len(external_train))
external_train.head()

6615


,context,question,answer_text,answer_start,language
0,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,एक प्रकार की रस्सी का नाम बताईए जो सेलेरी के क...,कोलेन्काइमा,2267,hindi
1,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,क्या चीज पूरी तरह से पैरेनकाइमा कोशिकाओं से बन...,पत्तियां,158,hindi
2,जाइलेम कोशिकाएं कोशिका भित्तियों की लिग्निकृत ...,जाइलेम किस प्रकार के पौधों में पाया जाता है?,ट्रेकियोफाइटों,292,hindi
3,फ्लोएम उच्चतर पौधों में आहार का संवहन करने वाल...,लेप्टोम क्या है?\n,समान कार्य करने वाला एक अधिक सरल ऊतक,969,hindi
4,"सभी हवाई अवयवों की बाह्यत्वचा, जड़ों को छोड़कर...",बाह्यत्वचा की कोशिकाएं क्या कर सकती हैं?,क्यटिन का संश्लेषण,433,hindi


In [3]:
external_train.answer_start.value_counts()

0       348
9        34
18       30
24       28
27       27
       ... 
2343      1
2439      1
4520      1
2507      1
2043      1
Name: answer_start, Length: 1258, dtype: int64

In [4]:
external_train.language.value_counts()

hindi    6615
Name: language, dtype: int64

In [5]:
train = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')#[:64]
print(len(train))
train.head()

1114


,id,context,question,answer_text,answer_start,language
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil


In [6]:
train.answer_start.value_counts()

0        76
16       10
58        8
12        7
21        7
         ..
481       1
16863     1
2526      1
477       1
62        1
Name: answer_start, Length: 723, dtype: int64

In [7]:
train.language.value_counts()

hindi    746
tamil    368
Name: language, dtype: int64

# Creating folds

In [8]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
folds = 5
kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
df = train #[:16]
for f, (t_, v_) in enumerate(kf.split(X=df, y=df.language.values)):
        df.loc[v_, 'kfold'] = f
external_train["kfold"] = -1
external_train['kfold'] = external_train['kfold'].astype('float64')
external_train['id'] = list(np.arange(1, len(external_train)+1))
external_train['id'] = external_train['id'].astype('object')
print(len(external_train))
print(len(df))
df.head()

6615
1114


,id,context,question,answer_text,answer_start,language,kfold
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,2.0
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,0.0
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil,0.0
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil,4.0
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil,0.0


In [9]:
# external_train.dtypes

In [10]:
# df.dtypes

In [11]:
def convert_answers(r):
    start = int(r[0])
    text = r[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

In [12]:
external_train['answers'] = external_train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)
external_train.head()

,context,question,answer_text,answer_start,language,kfold,id,answers
0,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,एक प्रकार की रस्सी का नाम बताईए जो सेलेरी के क...,कोलेन्काइमा,2267,hindi,-1.0,1,"{'answer_start': [2267], 'text': ['कोलेन्काइमा']}"
1,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,क्या चीज पूरी तरह से पैरेनकाइमा कोशिकाओं से बन...,पत्तियां,158,hindi,-1.0,2,"{'answer_start': [158], 'text': ['पत्तियां']}"
2,जाइलेम कोशिकाएं कोशिका भित्तियों की लिग्निकृत ...,जाइलेम किस प्रकार के पौधों में पाया जाता है?,ट्रेकियोफाइटों,292,hindi,-1.0,3,"{'answer_start': [292], 'text': ['ट्रेकियोफाइट..."
3,फ्लोएम उच्चतर पौधों में आहार का संवहन करने वाल...,लेप्टोम क्या है?\n,समान कार्य करने वाला एक अधिक सरल ऊतक,969,hindi,-1.0,4,"{'answer_start': [969], 'text': ['समान कार्य क..."
4,"सभी हवाई अवयवों की बाह्यत्वचा, जड़ों को छोड़कर...",बाह्यत्वचा की कोशिकाएं क्या कर सकती हैं?,क्यटिन का संश्लेषण,433,hindi,-1.0,5,"{'answer_start': [433], 'text': ['क्यटिन का सं..."


In [13]:
df['answers'] = df[['answer_start', 'answer_text']].apply(convert_answers, axis=1)
df.head()

,id,context,question,answer_text,answer_start,language,kfold,answers
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,2.0,"{'answer_start': [53], 'text': ['206']}"
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,0.0,"{'answer_start': [2358], 'text': ['காசுமீரில்']}"
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil,0.0,"{'answer_start': [0], 'text': ['சர் அலெக்ஸாண்ட..."
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil,4.0,"{'answer_start': [68], 'text': ['தாலாட்டு']}"
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil,0.0,"{'answer_start': [585], 'text': ['சூரியனும்']}"


In [14]:
# df.dtypes

In [15]:
# train.to_csv('train.csv',index=False)

# Loading custom data in hugging face datasets lib

In [16]:
# ! pip install datasets transformers
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cloud 0.1.13 requires tensorflow<3.0,>=1.15.0, which is not installed.
dask-cudf 21.6.1+2.g101fc0fda4 requires cupy-cuda112, which is not installed.
cudf 21.6.1+2.g101fc0fda4 requires cupy-cuda110, which is not installed.
s3fs 2021.6.1 requires fsspec==2021.06.1, but you have fsspec 2021.6.0 which is incompatible.
pytorch-lightning 1.3.8 requires fsspec[http]!=2021.06.0,>=2021.05.0, but you have fsspec 2021.6.0 which is incompatible.
dask-cudf 21.6.1+2.g101fc0fda4 requires dask<=2021.5.1,>=2021.4.0, but you have dask 2021.6.2 which is incompatible.
dask-cudf 21.6.1+2.g101fc0fda4 requires distributed<=2021.5.1,>=2.22.0, but you have distributed 2021.6.2 which is incompatible.


In [17]:
from datasets import load_dataset, Dataset
from pprint import pprint

# Preprocessing

In [18]:
# model_checkpoint = "distilbert-base-uncased"
# model_checkpoint = "deepset/xlm-roberta-large-squad2"
# model_checkpoint = "../input/xlm-roberta-squad2/deepset/xlm-roberta-base-squad2"
model_checkpoint = "../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2"

from transformers import XLMTokenizer,AutoTokenizer
# tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Doc_stride is used to handle large text: tokens>512

In [19]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

# putting all together

In [20]:
# In some padding required on left side
pad_on_right = tokenizer.padding_side == "right"

In [21]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")
#     print(len(offset_mapping))

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [22]:
# dataset = Dataset.from_pandas(train[:-64])
# eval_dataset = Dataset.from_pandas(train[-64:])

#just to verify pipeline
# dataset = Dataset.from_pandas(train[:32])
# eval_dataset = Dataset.from_pandas(train[32:48])

In [23]:
# tokenized_train_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=dataset.column_names)
# tokenized_eval_datasets = eval_dataset.map(prepare_train_features, batched=True, remove_columns=eval_dataset.column_names)

# Finetune start

In [24]:
import gc

from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

data_collator = default_data_collator
gc.collect()
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [25]:
def chaiiModel(model_checkpoint):
     return AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [26]:
def getTrainArgsForExData(model_checkpoint):
    # CHnage batch size and gradient accumulation
    batch_size = 4#8#16
    model_name = model_checkpoint.split("/")[-1]
    return TrainingArguments(
        f"test-chaii",
#         overwrite_output_dir=True,
        save_total_limit = 1,
        save_strategy = "no",
#         logging_strategy = "no",
        
    #     evaluation_strategy = "epoch",
    #     save_strategy = "epoch",
    #     learning_rate=3e-5,
    #     warmup_ratio=0.1,
    #     gradient_accumulation_steps=8,
    #     per_device_train_batch_size=batch_size,
    #     per_device_eval_batch_size=batch_size,
    #     num_train_epochs=3,
    #     weight_decay=0.01,

        evaluation_strategy = "epoch",
        learning_rate=1e-5,#2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=1,#1,# make it 3
        gradient_accumulation_steps=8,#4,
        weight_decay=0.01,
    )

In [27]:
def getTrainArgs(model_checkpoint):
    # CHnage batch size and gradient accumulation
    batch_size = 4#8#16
    model_name = model_checkpoint.split("/")[-1]
    return TrainingArguments(
        f"test-chaii",
#         overwrite_output_dir=True,
        save_total_limit = 1,
        save_strategy = "no",
#         logging_strategy = "no",
        
    #     evaluation_strategy = "epoch",
    #     save_strategy = "epoch",
    #     learning_rate=3e-5,
    #     warmup_ratio=0.1,
    #     gradient_accumulation_steps=8,
    #     per_device_train_batch_size=batch_size,
    #     per_device_eval_batch_size=batch_size,
    #     num_train_epochs=3,
    #     weight_decay=0.01,

        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=2,#1,# make it 3
        gradient_accumulation_steps=8,#4,
        weight_decay=0.01,
    )

In [28]:
def getData(df, fold):
    train = df[df['kfold']!=fold]
    valid = df[df['kfold']==fold]
    dataset = Dataset.from_pandas(train)
    eval_dataset = Dataset.from_pandas(valid)
    tokenized_train_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=dataset.column_names)
    tokenized_eval_datasets = eval_dataset.map(prepare_train_features, batched=True, remove_columns=eval_dataset.column_names)
    return tokenized_train_datasets, tokenized_eval_datasets

In [29]:
args = getTrainArgs(model_checkpoint)
ex_data_args =  getTrainArgs(model_checkpoint)
for fold in range(folds):
        print(f'Preparing data for fold number {fold}')
        tokenized_External_train_datasets, _ = getData(external_train, fold)
        print('tokenized_External_train_datasets: ',len(tokenized_External_train_datasets))
        tokenized_train_datasets, tokenized_eval_datasets = getData(df, fold)
        print('tokenized_train_datasets: ',len(tokenized_train_datasets))
        print('tokenized_eval_datasets: ',len(tokenized_eval_datasets))
        print(f'model {fold} loading ...')
        model = chaiiModel(model_checkpoint)
        print(f'training with externa data fold: {fold}')
        trainer = Trainer(
                            model,
                            ex_data_args,
                            train_dataset=tokenized_External_train_datasets,
                            eval_dataset=tokenized_eval_datasets,
                            data_collator=data_collator,
                            tokenizer=tokenizer,
                        )
        
        trainer.train()
        print(f'training with actual data fold: {fold}')
        trainer = Trainer(
                            model,
                            args,
                            train_dataset=tokenized_train_datasets,
                            eval_dataset=tokenized_eval_datasets,
                            data_collator=data_collator,
                            tokenizer=tokenizer,
                        )
        
        trainer.train()
        
        trainer.save_model(f"chaii-trained-model-{fold}")
        print(f"Training fold {fold}")
        print("----------------------")

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Preparing data for fold number 0



tokenized_External_train_datasets:  8328



tokenized_train_datasets:  11875
tokenized_eval_datasets:  2866
model 0 loading ...
training with externa data fold: 0


Epoch,Training Loss,Validation Loss
0,No log,0.446898
1,1.115700,0.485036


...


tokenized_train_datasets:  11919
tokenized_eval_datasets:  2822
model 4 loading ...
training with externa data fold: 4


Epoch,Training Loss,Validation Loss
0,No log,0.530206
1,1.115700,0.611040


training with actual data fold: 4


Epoch,Training Loss,Validation Loss
0,No log,0.238945
1,0.223000,0.291764


Training fold 4
----------------------
